# Abuse and punishment

Querying Feb. 1 data for all violations involving abuse and punishment for background information.

In [1]:
import pandas as pd

In [2]:
non_compliance = pd.read_csv('../csv/dfps-2018-02-01/non_compliance.csv').drop('Unnamed: 0', axis=1)
operations = pd.read_csv('../csv/dfps-2018-02-01/operations.csv').drop('Unnamed: 0', axis=1)
inspections = pd.read_csv('../csv/dfps-2018-02-01/assessment.csv').drop('Unnamed: 0', axis=1)

In [3]:
abuse_punishment = non_compliance.assign(
    cruel = lambda x: x.STANDARD_NUMBER_DESCRIPTION.str.lower().isin(
        pd.Series(list(filter(
            lambda x: 'harsh, cruel' in x,
            [a for a in non_compliance.STANDARD_NUMBER_DESCRIPTION.str.lower().unique()]
        )))
    ) | x.STANDARD_NUMBER_DESCRIPTION.isin([
        '746.2805(4) - Prohibited Punishments - Hitting with Hand or Instrument',
        '746.2805(3) - Prohibited Punishments - Pinching, Shaking, or Biting',
        '746.2805(8) - Prohibited Punishments -Placing Child in Locked, Dark Room'
    ])
).merge(
    operations,
    on='OPERATION_ID'
).query(
    'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
).query('cruel == True').append(
    non_compliance.assign(
        cruel = lambda x: x.STANDARD_NUMBER_DESCRIPTION.str.lower().isin(
            pd.Series(list(filter(
                lambda x: 'neglected' in x,
                [a for a in non_compliance.STANDARD_NUMBER_DESCRIPTION.str.lower().unique()]
            )))
        )
    ).merge(
        operations,
        on='OPERATION_ID'
    ).query(
        'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
    ).query('cruel == True').set_index(
        'ACTIVITY_ID'
    )
)[[
    'OPERATION_NAME', 'LOCATION_ADDRESS', 'COUNTY',
    'STANDARD_NUMBER_DESCRIPTION',
    'CORRECTED_DATE',
    'NARRATIVE', 'TECHNICAL_ASSISTANCE_GIVEN', 'OPERATION_TYPE',
]].assign(
    in_timeframe = lambda x: pd.to_datetime(x.CORRECTED_DATE).apply(
        lambda x: x >= pd.to_datetime('2016-02-01')
    )
).query('in_timeframe').drop(
    'in_timeframe', axis=1
)

In [4]:
abuse_punishment.to_csv(
    '../export/abuse_punishment.csv',
    index=False
)